In [1]:
import sys
#sys.path.append("../usr/lib/")

In [2]:
INPUT_ROOT = '/home/cszsolnai/Projects/kaggle_covid19_nlp/input'
OUTPUT_ROOT = '/home/cszsolnai/Projects/kaggle_covid19_nlp/output'
BIOBERT_ROOT = '/home/cszsolnai/Projects/kaggle_covid19_nlp/input/biobert_v1.1_pubmed'

METADATA = INPUT_ROOT + '/metadata.csv'
MODEL_PATH = BIOBERT_ROOT + '/model.ckpt-1000000'
VOCAB_FILE = BIOBERT_ROOT + '/vocab.txt'
BERT_CONFIG = BIOBERT_ROOT + '/bert_config.json'

In [3]:
#!pip install tensorflow-gpu==1.14.0
#!pip install bert-tensorflow

In [4]:
import pickle
import argparse
import re
import os
import glob
import json
import pandas as pd
from tqdm.notebook import tqdm
import nltk
import argparse
import re
import numpy as np
import pickle
from tqdm import tqdm
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.decomposition import PCA
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords

In [6]:
from bert_embeddings import bert_embedding_generator

/home/cszsolnai/anaconda3/envs/nlp_tf1/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/cszsolnai/anaconda3/envs/nlp_tf1/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/cszsolnai/anaconda3/envs/nlp_tf1/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/cszsolnai/anaconda3/envs/nlp

In [7]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/cszsolnai/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [17]:
def fold(e):
    if isinstance(e, list):
        return ' '.join(e)
    else:
        return e

def get_content(file_path):
    with open(file_path) as file:
        content = json.load(file)

        if 'abstract' in content:
            abstract = []
            for entry in content['abstract']:
                abstract.append(entry['text'])

            full_abstract = '\n'.join(abstract)
        else:
            full_abstract = ''

        # Body text
        articles_text = []
        if 'body_text' in content:
            for entry in content['body_text']:
                articles_text.append(entry['text'])
            full_text = '\n'.join(articles_text)
            concat_text = '\n'.join([full_abstract, full_text]).strip()
        else:
            full_text = ''
            concat_text = ''
            


        # Authors
        authors = []
        if 'authors' in content['metadata']:
            for author in content['metadata']['authors']:
                authors.append(' '.join([fold(author['first']), fold(author['middle']), fold(author['last'])]))

        return content['paper_id'], concat_text, content['metadata'][
            'title'], full_abstract, '\n'.join(authors), full_text


def lower_case(input_str):
    input_str = input_str.lower()
    return input_str

Save all articles as dataframe

In [5]:
all_json = glob.glob(INPUT_ROOT + '/**/*.json', recursive=True)

In [15]:
# https://www.kaggle.com/allen-institute-for-ai/CORD-19-research-challenge/tasks?taskId=887
query_text = "What is the best method to combat the hypercoagulable state seen in COVID-19 ?"

In [16]:
converted_query = next(bert_embedding_generator([query_text], [0], BERT_CONFIG, VOCAB_FILE, MODEL_PATH))
query_embeddings = np.stack([e[0] for e in converted_query.values()])


The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmp709og5u5', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': None, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_service': None, '_cluster_spec': <tensorflow.python.trai

In [23]:
# Get the article with the closest embedding tot the query string embedding


class EmptyContent(Exception):
    pass


def flatten_list(values):
    l = []
    for e in values:
        if isinstance(e, list):
               for embedding in e:
                    l.append(embedding)
        else:
            l.append(e)
    return l


def average_query_distance(article_embeddings):
    avg_best_distance_from_query = 0
    for i in range(query_embeddings.shape[0]):
        avg_best_distance_from_query += np.min(np.linalg.norm(query_embeddings[i] - article_embeddings, axis=1))
    return avg_best_distance_from_query / query_embeddings.shape[0]


def article_content_generator(all_json):
    for idx, entry in tqdm(enumerate(all_json)):
        paper_id, concat_text, title, abstract, authors, _ = get_content(entry)
        if concat_text == '':
            yield EmptyContent()
        else:
            yield concat_text


closest_ids = []
closest_num = 10
closest_distances = []

update_frequency = 1000

for idx, converted_article in enumerate(bert_embedding_generator(article_content_generator(all_json), list(range(len(all_json))), BERT_CONFIG, VOCAB_FILE, MODEL_PATH)):
    if isinstance(converted_article, EmptyContent):
        continue
    
    article_embeddings = np.stack(flatten_list(converted_article.values()))
            
    closest_ids.append(idx)
    closest_distances.append(average_query_distance(article_embeddings))
    
    # Remove article id with the highest distance
    if len(closest_ids) > closest_num:
        max_id = np.argmax(closest_distances)
        del closest_distances[max_id]
        del closest_ids[max_id]

    if (idx+1) % update_frequency == 0:
        with open('checkpoint.pkl', 'wb') as f:
            data = {
                'idx': idx,
                'closest_ids': closest_ids,
                'closest_distances': closest_distances
            }
            pickle.dump(data, f)




INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmps53fgrvp', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': None, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f6efc58e6a0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1, '_tpu_config': TPUConfig(iterations_per_loop=2, num_shards=8, num_cores_per_replica=None, per_host_input_for_training=3, tpu_job_name

1023it [06:07,  2.78it/s]


INFO:tensorflow:**** Trainable Variables ****
INFO:tensorflow:  name = bert/embeddings/word_embeddings:0, shape = (28996, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/embeddings/token_type_embeddings:0, shape = (2, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/embeddings/position_embeddings:0, shape = (512, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/embeddings/LayerNorm/beta:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/embeddings/LayerNorm/gamma:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_0/attention/self/query/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_0/attention/self/query/bias:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_0/attention/self/key/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_0/attention/self/key/bias:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = ber

0it [00:00, ?it/s]

INFO:tensorflow:*** Example ***
INFO:tensorflow:unique_id: 0
INFO:tensorflow:tokens: [CLS] the appearance , disappearance , and re - emergence of novel path ##ogen ##s of humans with both epidemic potential and high mortality rates have threatened global health security for centuries . 1 over the past few decades , notable new zoo ##not ##ic infectious diseases of humans have been caused by path ##ogen ##s arising from animal reservoirs , including west ni ##le virus , yellow fever virus , e ##bol ##a virus , ni ##pa ##h virus , las ##sa fever virus , ha ##nta ##virus , den ##gue virus , rift valley fever virus , crime ##an - con ##go ha ##em ##or ##r ##ha ##gic fever virus , severe acute respiratory syndrome co ##rona ##virus ( sa ##rs - co ##v ) [SEP]
INFO:tensorflow:input_ids: 101 1103 2468 117 14158 117 1105 1231 118 15351 1104 2281 3507 19790 1116 1104 3612 1114 1241 18162 3209 1105 1344 14471 5600 1138 4963 4265 2332 2699 1111 3944 119 122 1166 1103 1763 1374 4397 117 3385 1207 1

...

4it [00:00, 16.44it/s]

INFO:tensorflow:*** Example ***
INFO:tensorflow:unique_id: 4
INFO:tensorflow:tokens: [CLS] les virus re ##sp ##ira ##toire ##s son ##t re ##sp ##ons ##ables d ' en ##vir ##on 80 % des infections re ##sp ##ira ##toire ##s . il ##s son ##t re ##sp ##ons ##ables d ' infections re ##sp ##ira ##toire ##s ha ##utes ( r ##hin ##ite , la ##ryn ##got ##rac ##he ##ite ) ma ##is au ##ssi d ' infections re ##sp ##ira ##toire ##s bass ##es ( br ##on ##chi ##te , br ##on ##chi ##oli ##te , p ##ne ##um ##op ##ath ##ie ) potent ##iel ##lement severe ##s . l ' incidence an ##nu ##elle des p ##ne ##um ##onies com ##mu ##na ##uta ##ires ch ##ez l ' en ##fant de m ##oi ##ns de c ##in [SEP]
INFO:tensorflow:input_ids: 101 8241 7942 1231 20080 5132 19101 1116 1488 1204 1231 20080 4199 15759 173 112 4035 25740 1320 2908 110 3532 16565 1231 20080 5132 19101 1116 119 14596 1116 1488 1204 1231 20080 4199 15759 173 112 16565 1231 20080 5132 19101 1116 5871 20311 113 187 8265 3150 117 2495 15023 18855 19366 4638 3

120464it [5:17:15,  6.33it/s]


ERROR:tensorflow:Error recorded from prediction_loop: 2 root error(s) found.
  (0) Unknown: KeyError: 'body_text'
Traceback (most recent call last):

  File "/home/cszsolnai/anaconda3/envs/nlp_tf1/lib/python3.6/site-packages/tensorflow/python/ops/script_ops.py", line 209, in __call__
    ret = func(*args)

  File "/home/cszsolnai/anaconda3/envs/nlp_tf1/lib/python3.6/site-packages/tensorflow/python/data/ops/dataset_ops.py", line 514, in generator_py_func
    values = next(generator_state.get_iterator(iterator_id))

  File "/home/cszsolnai/Projects/kaggle_covid19_nlp/src/bert_embeddings.py", line 71, in input_generator
    for instance_features in features:

  File "/home/cszsolnai/Projects/kaggle_covid19_nlp/src/bert_embeddings.py", line 171, in features_generator
    for (ex_index, example) in enumerate(full_texts):

  File "<ipython-input-23-8f8e98a9f332>", line 25, in article_content_generator
    paper_id, concat_text, title, abstract, authors, _ = get_content(entry)

  File "<ipyth

UnknownError: 2 root error(s) found.
  (0) Unknown: KeyError: 'body_text'
Traceback (most recent call last):

  File "/home/cszsolnai/anaconda3/envs/nlp_tf1/lib/python3.6/site-packages/tensorflow/python/ops/script_ops.py", line 209, in __call__
    ret = func(*args)

  File "/home/cszsolnai/anaconda3/envs/nlp_tf1/lib/python3.6/site-packages/tensorflow/python/data/ops/dataset_ops.py", line 514, in generator_py_func
    values = next(generator_state.get_iterator(iterator_id))

  File "/home/cszsolnai/Projects/kaggle_covid19_nlp/src/bert_embeddings.py", line 71, in input_generator
    for instance_features in features:

  File "/home/cszsolnai/Projects/kaggle_covid19_nlp/src/bert_embeddings.py", line 171, in features_generator
    for (ex_index, example) in enumerate(full_texts):

  File "<ipython-input-23-8f8e98a9f332>", line 25, in article_content_generator
    paper_id, concat_text, title, abstract, authors, _ = get_content(entry)

  File "<ipython-input-17-ff08f1b745f4>", line 22, in 

In [6]:
with open('checkpoint.pkl', 'rb') as f:
    data = pickle.load(f)
    idx = data['idx']
    closest_ids = data['closest_ids']
    closest_distances = data['closest_distances']

In [7]:
idxs = np.argsort(closest_distances)
closest_ids = np.array(closest_ids)[idxs]
closest_distances = np.array(closest_distances)[idxs]

selected_jsons = list(np.array(all_json)[idxs])

#selected_jsons = [all_json[0]]

In [8]:
import torch
from torch.utils.data import DataLoader, SequentialSampler
from collections import namedtuple
from transformers import BertTokenizer
from modeling_bertabs import BertAbs, build_predictor

from utils_bertabs import (
    CovidDataset,
    build_mask,
    compute_token_type_ids,
    encode_for_summarization,
    truncate_or_pad,
)

/home/cszsolnai/anaconda3/envs/nlp_tf1/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/cszsolnai/anaconda3/envs/nlp_tf1/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/cszsolnai/anaconda3/envs/nlp_tf1/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/cszsolnai/anaconda3/envs/nlp

In [9]:
def build_data_iterator(all_jsons, args, tokenizer):
    dataset = CovidDataset(all_jsons)
    sampler = SequentialSampler(dataset)

    def collate_fn(data):
        return collate(data, tokenizer, block_size=512, device=args['device'])

    iterator = DataLoader(dataset, sampler=sampler, batch_size=args['batch_size'], collate_fn=collate_fn, )

    return iterator, dataset


def collate(data, tokenizer, block_size, device):
    """ Collate formats the data passed to the data loader.

    In particular we tokenize the data batch after batch to avoid keeping them
    all in memory. We output the data as a namedtuple to fit the original BertAbs's
    API.
    """
    data = [x for x in data if not len(x[1]) == 0]  # remove empty_files
    names = [name for name, _, _ in data]
    summaries = [" ".join(summary_list) for _, _, summary_list in data]

    encoded_text = [encode_for_summarization(story, summary, tokenizer) for _, story, summary in data]
    encoded_stories = torch.tensor(
        [truncate_or_pad(story, block_size, tokenizer.pad_token_id) for story, _ in encoded_text]
    )
    encoder_token_type_ids = compute_token_type_ids(encoded_stories, tokenizer.cls_token_id)
    encoder_mask = build_mask(encoded_stories, tokenizer.pad_token_id)

    batch = Batch(
        document_names=names,
        batch_size=len(encoded_stories),
        src=encoded_stories.to(device),
        segs=encoder_token_type_ids.to(device),
        mask_src=encoder_mask.to(device),
        tgt_str=summaries,
    )

    return batch


In [10]:
Batch = namedtuple("Batch", ["document_names", "batch_size", "src", "segs", "mask_src", "tgt_str"])

BATCH_SIZE = 1
MIN_LENGTH = 50
MAX_LENGTH = 200
BEAM_SIZE = 5
DEVICE = torch.device("cuda")

args = {
    'documents_dir': INPUT_ROOT,
    'summaries_output_dir': 'output_dir',
    'compute_rouge': False,
    'no_cuda': False,
    'batch_size': BATCH_SIZE,
    'min_length': MIN_LENGTH,
    'max_length': MAX_LENGTH,
    'beam_size': BEAM_SIZE,
    'alpha': 0.95,
    'block_trigram': True,
    'device': DEVICE
}

In [11]:
df = pd.read_csv(
    METADATA,
    usecols=["title", "abstract", "authors", "doi", "publish_time"],
)

# drop duplicates
# df=df.drop_duplicates()
df = df.drop_duplicates(subset="abstract", keep="first")
# drop NANs
df = df.dropna()

df.head()

,title,doi,abstract,publish_time,authors
0,Clinical features of culture-proven Mycoplasma...,10.1186/1471-2334-1-6,OBJECTIVE: This retrospective chart review des...,2001-07-04,"Madani, Tariq A; Al-Ghamdi, Aisha A"
1,Nitric oxide: a pro-inflammatory mediator in l...,10.1186/rr14,Inflammatory diseases of the respiratory tract...,2000-08-15,"Vliet, Albert van der; Eiserich, Jason P; Cros..."
2,Surfactant protein-D and pulmonary host defense,10.1186/rr19,Surfactant protein-D (SP-D) participates in th...,2000-08-25,"Crouch, Erika C"
3,Role of endothelin-1 in lung disease,10.1186/rr44,Endothelin-1 (ET-1) is a 21 amino acid peptide...,2001-02-22,"Fagan, Karen A; McMurtry, Ivan F; Rodman, David M"
4,Gene expression in epithelial cells in respons...,10.1186/rr61,Respiratory syncytial virus (RSV) and pneumoni...,2001-05-11,"Domachowske, Joseph B; Bonville, Cynthia A; Ro..."


In [12]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased", do_lower_case=True)

model = BertAbs.from_pretrained("bertabs-finetuned-cnndm")
model.to(args['device'])
model.eval()

symbols = {
    "BOS": tokenizer.vocab["[unused0]"],
    "EOS": tokenizer.vocab["[unused1]"],
    "PAD": tokenizer.vocab["[PAD]"],
}

args['result_path'] = ""
args['temp_dir'] = ""

data_iterator, dataset = build_data_iterator(selected_jsons, args, tokenizer)

predictor = build_predictor(args, tokenizer, symbols, model)

In [13]:
all_summaries =  []

data = {'Paper id': [], 'Title': [], 'Summary': []}

for i, batch in enumerate(tqdm(data_iterator)):
    article = selected_jsons[i]
    
    batch_data = predictor.translate_batch(batch)
    translations = predictor.from_batch(batch_data)
    data['Paper id'].append(article['metadata']['paper_id'])
    data['Title'].append(article['metadata']['title'])
    data['Summary'].append(translations[0][2])
df = pd.DataFrame.from_dict(data)
df.to_csv(os.path.join(OUTPUT_ROOT, 'summaries.csv'))

  0%|          | 0/10 [00:01<?, ?it/s]


TypeError: string indices must be integers